***GENERATED CODE FOR 70cols PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern', 'transformation_label': 'String Indexer'}], 'feature': 'Intern', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '416', 'mean': '0.0', 'stddev': 'nan', 'min': '0', 'max': 'yonas tsegay', 'missing': '0', 'distinct': '416'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern'}, {'feature_label': 'Intern', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternStartDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternStartDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '416', 'mean': '', 'stddev': '', 'min': '1/10/2022', 'max': '9/6/2022', 'missing': '0', 'distinct': '184'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternStartDate'}, {'feature_label': 'InternStartDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternStartDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternEndDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '416', 'mean': '', 'stddev': '', 'min': '1/10/2022', 'max': '9/9/2020', 'missing': '0', 'distinct': '249'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternEndDate'}, {'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternEndDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_PlacementDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '416', 'mean': '0.0', 'stddev': '0.0', 'min': '0', 'max': '8/15/2023', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_PlacementDate'}, {'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_PlacementDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReportDate', 'transformation_label': 'String Indexer'}], 'feature': 'ReportDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '416', 'mean': '', 'stddev': '', 'min': '1/12/2021', 'max': 'Unknown', 'missing': '0', 'distinct': '165'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReportDate'}, {'feature_label': 'ReportDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReportDate')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 70colsHooks.ipynb
try:
	#sourcePreExecutionHook()

	forgepreproceeddatanjyn = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1724854257541_ForgePreprocessed_data70_1.csv', 'filename': 'ForgePreprocessed_data70_1.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/clbr bug/ForgePreprocessed_data70_1.csv', 'viewFileName': 'ForgePreprocessed_data70_1.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 70colsHooks.ipynb
try:
	#transformationPreExecutionHook()

	colsautofe = TransformationMain.run(forgepreproceeddatanjyn,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Intern", "transformation_label": "String Indexer"}], "feature": "Intern", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "416", "mean": "0.0", "stddev": "nan", "min": "0", "max": "yonas tsegay", "missing": "0", "distinct": "416"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern"}, {"transformationsData": [{"feature_label": "InternStartDate", "transformation_label": "String Indexer"}], "feature": "InternStartDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "416", "mean": "", "stddev": "", "min": "1/10/2022", "max": "9/6/2022", "missing": "0", "distinct": "184"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternStartDate"}, {"transformationsData": [{"feature_label": "InternEndDate", "transformation_label": "String Indexer"}], "feature": "InternEndDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "416", "mean": "", "stddev": "", "min": "1/10/2022", "max": "9/9/2020", "missing": "0", "distinct": "249"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternEndDate"}, {"transformationsData": [{"feature_label": "Intern_MM_PlacementDate", "transformation_label": "String Indexer"}], "feature": "Intern_MM_PlacementDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "416", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "8/15/2023", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PlacementDate"}, {"transformationsData": [{"feature_label": "ReportDate", "transformation_label": "String Indexer"}], "feature": "ReportDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "416", "mean": "", "stddev": "", "min": "1/12/2021", "max": "Unknown", "missing": "0", "distinct": "165"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReportDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ClassSignupsID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "10044.92", "stddev": "2641.13", "min": "2469", "max": "18246", "missing": "0"}, "updatedLabel": "ClassSignupsID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysInInternship", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "164.48", "stddev": "176.16", "min": "1", "max": "1760", "missing": "0"}, "updatedLabel": "DaysInInternship"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveySent", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.22", "stddev": "0.64", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveySent"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INTERNSHIP_SCORE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.29", "stddev": "0.1", "min": "0.25", "max": "0.817422231", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INTERNSHIP_SCORE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysTaken", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.12", "stddev": "0.45", "min": "0.0", "max": "3.428571429", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveysTaken"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysGiven", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.11", "stddev": "0.4", "min": "0.0", "max": "2.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveysGiven"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternCancelReasonID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "1.89", "stddev": "1.49", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "InternCancelReasonID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QuestionsScored", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "1.41", "stddev": "5.14", "min": "0.0", "max": "36.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "QuestionsScored"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WorkEthic", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "WorkEthic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Availability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Availability"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TeamPlayer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TeamPlayer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QualityofWork", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "QualityofWork"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Trust_2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Trust_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Respectful", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.06", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Respectful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveyScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveyScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.04", "stddev": "0.13", "min": "0.0", "max": "0.794871795", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.06", "stddev": "0.17", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.04", "stddev": "0.13", "min": "0.0", "max": "0.794871795", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Attendance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.1", "stddev": "0.39", "min": "0.0", "max": "2.846153846", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Attendance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampCommentScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.1", "stddev": "0.39", "min": "0.0", "max": "2.846153846", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCampCommentScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampTotalComments", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.16", "stddev": "0.64", "min": "0.0", "max": "5.871794872", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCampTotalComments"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.06", "stddev": "0.17", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternTechGroupID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "3.9", "stddev": "2.66", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "InternTechGroupID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Internquit", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "0.25", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Internquit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_WkOrder", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "12.67", "stddev": "29.31", "min": "1.0", "max": "145.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_WkOrder"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "3.89", "stddev": "8.93", "min": "0.0", "max": "41.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Yr", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "379.27", "stddev": "790.46", "min": "0.0", "max": "2024.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Yr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_HW", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.01", "stddev": "0.06", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_HW"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Videos", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.02", "stddev": "0.1", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Videos"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_VideosScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.07", "stddev": "0.36", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_VideosScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoInt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.1", "stddev": "0.4", "min": "0.0", "max": "4.25", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_AutoInt"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Phone", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.03", "stddev": "0.13", "min": "0.0", "max": "1.333333333", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Phone"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_LoginPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.11", "stddev": "0.25", "min": "0.0", "max": "0.989230769", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_LoginPer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.22", "stddev": "0.63", "min": "0.0", "max": "5.75", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ActivityScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ParticipationPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.1", "stddev": "0.25", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ParticipationPe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserComments", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.13", "stddev": "0.38", "min": "0.0", "max": "3.153846154", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_UserComments"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScoreChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.1", "stddev": "0.25", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_ActivityScoreCh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserCommentsChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.11", "stddev": "0.28", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_UserCommentsChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_JRPChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Intern_MM_JRPChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_SubChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Intern_MM_SubChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoIntChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.05", "stddev": "0.16", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_AutoIntChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_PhoneChk", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.02", "stddev": "0.1", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PhoneChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.04", "stddev": "0.09", "min": "0.0", "max": "0.39229125", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk_ActivityScor..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_Answer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.23", "stddev": "1.29", "min": "0.0", "max": "18.45714286", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_Answer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_CorrectAnswer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.0", "stddev": "0.02", "min": "0.0", "max": "0.222222222", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_CorrectAnswer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_VideosHosted", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.047619048", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_VideosHosted"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_QualfyVideo", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.047619048", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_CC_QualfyVideo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_Participation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.49", "stddev": "1.59", "min": "0.0", "max": "13.92307692", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_COE_Participation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_ProjectUpdates", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.09", "stddev": "0.45", "min": "0.0", "max": "5.583333333", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_COE_ProjectUpdates"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.22", "stddev": "0.62", "min": "0.0", "max": "5.75", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.2", "stddev": "0.93", "min": "0.0", "max": "8.171428571", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.56", "stddev": "1.63", "min": "0.0", "max": "9.923076923", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Part3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score_Pre_Total", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "0.97", "stddev": "2.81", "min": "0.0", "max": "20.25", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score_Pre_Total"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternLevelGroup", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "2.44", "stddev": "1.03", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "InternLevelGroup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dominance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "20.83", "stddev": "23.42", "min": "0", "max": "92", "missing": "0"}, "updatedLabel": "Dominance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Influencer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "29.01", "stddev": "29.42", "min": "0", "max": "88", "missing": "0"}, "updatedLabel": "Influencer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Steadiness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "36.64", "stddev": "35.37", "min": "0", "max": "96", "missing": "0"}, "updatedLabel": "Steadiness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Compliance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "33.31", "stddev": "32.79", "min": "0", "max": "94", "missing": "0"}, "updatedLabel": "Compliance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Theoretical", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "28.64", "stddev": "26.62", "min": "0", "max": "85", "missing": "0"}, "updatedLabel": "Theoretical"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Utilitarian", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "23.51", "stddev": "22.24", "min": "0", "max": "63", "missing": "0"}, "updatedLabel": "Utilitarian"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Aesthetic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "21.48", "stddev": "20.3", "min": "0", "max": "66", "missing": "0"}, "updatedLabel": "Aesthetic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Social", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "23.99", "stddev": "23.35", "min": "0", "max": "70", "missing": "0"}, "updatedLabel": "Social"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Individualistic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "22.91", "stddev": "21.94", "min": "0", "max": "82", "missing": "0"}, "updatedLabel": "Individualistic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Traditional", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "416", "mean": "19.65", "stddev": "18.93", "min": "0", "max": "65", "missing": "0"}, "updatedLabel": "Traditional"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Flexibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "416", "mean": "3.96", "stddev": "4.47", "min": "0.0", "max": "52.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Flexibility"}]}))

	#transformationPostExecutionHook(colsautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 70colsHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(colsautofe, ["ClassSignupsID", "DaysInInternship", "SurveySent", "INTERNSHIP_SCORE", "SurveysTaken", "SurveysGiven", "InternCancelReasonID", "QuestionsScored", "WorkEthic", "Availability", "TeamPlayer", "QualityofWork", "Trust_2", "Respectful", "SurveyScore", "BaseCamp_FinalScore", "IPBC_FinalScore", "BaseCamp_Score", "Attendance", "BaseCampCommentScore", "BaseCampTotalComments", "IPBC_Score", "InternTechGroupID", "Intern_MM_WkOrder", "Intern_MM_Wk", "Intern_MM_Yr", "Intern_MM_HW", "Intern_MM_Videos", "Intern_MM_VideosScore", "Intern_MM_AutoInt", "Intern_MM_Phone", "Intern_MM_LoginPer", "Intern_MM_ActivityScore", "Intern_MM_ParticipationPer", "Intern_MM_UserComments", "Intern_MM_ActivityScoreChk", "Intern_MM_UserCommentsChk", "Intern_MM_JRPChk", "Intern_MM_SubChk", "Intern_MM_AutoIntChk", "Intern_MM_PhoneChk", "Intern_MM_Wk_ActivityScore", "Intern_CC_Answer", "Intern_CC_CorrectAnswer", "Intern_CC_VideosHosted", "Intern_CC_QualfyVideo", "Intern_COE_Participation", "Intern_COE_ProjectUpdates", "IPBC_Part1", "IPBC_Part2", "IPBC_Part3", "IPBC_Score_Pre_Total", "InternLevelGroup", "Dominance", "Influencer", "Steadiness", "Compliance", "Theoretical", "Utilitarian", "Aesthetic", "Social", "Individualistic", "Traditional", "Flexibility", "Intern_stringindexer", "InternStartDate_stringindexer", "InternEndDate_stringindexer", "Intern_MM_PlacementDate_stringindexer", "ReportDate_stringindexer"], "Internquit")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

